In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#import sframe
%matplotlib inline


In [3]:

loans = pd.read_csv('lending-club-data.csv',low_memory = False)

In [4]:
loans.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'is_inc_v', 'issue_d',
       'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title',
       'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'not_compliant', 'status', 'inactive_loans', 'bad_loans',
       'emp_length_num', 'grade_num', 'sub_gra

In [5]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
del loans['bad_loans']

In [6]:
len(loans[loans['safe_loans'] == 1])/len(loans)

0.8111853319957262

In [7]:
features = ['grade',                     # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]

target = 'safe_loans'                    # prediction target (y) (+1 means safe, -1 is risky)

# Extract the feature columns and target column
loans = loans[features + [target]]

In [20]:
import json
with open('module-5-assignment-1-train-idx.json') as json_data:
    train = json.load(json_data)
    
with open('module-5-assignment-1-validation-idx.json') as json_data:
    validation = json.load(json_data)

In [21]:
safe_loans_raw = loans[loans[target] == +1]
risky_loans_raw = loans[loans[target] == -1]
print ("Number of safe loans  : %s" % len(safe_loans_raw))
print ("Number of risky loans : %s" % len(risky_loans_raw))

Number of safe loans  : 99457
Number of risky loans : 23150


In [52]:
# Since there are fewer risky loans than safe loans, find the ratio of the sizes
# and use that percentage to undersample the safe loans.
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))

risky_loans = risky_loans_raw
safe_loans = safe_loans_raw.sample(frac = percentage,random_state = 1)

# Append the risky_loans with the downsampled version of safe_loans
loans_data = risky_loans.append(safe_loans)

In [53]:
def unpack(df, column,fillna=None):
    ret = None
    if fillna is None:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems()), index = df.index)], axis=1)
        del ret[column]
        print(type(ret.columns))
        ret.columns = str(column)+ret.columns
    else:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems()), index = df.index).fillna(fillna)], axis=1)
        del ret[column]
        ret.columns = str(column)+'.'+ret.columns
    return ret

In [64]:
#loans_data = risky_loans.append(safe_loans)
loans_data = loans
categorical_variables = []
for feat_name, feat_type in zip(loans_data.columns, loans_data.dtypes):
    if feat_type == object:
        categorical_variables.append(feat_name)

for feature in categorical_variables:
    loans_data_one_hot_encoded = loans_data[feature].apply(lambda x: {x: 1})
    
    df = pd.DataFrame(loans_data_one_hot_encoded)
    loans_data_unpacked = unpack(df,feature,0)
    #print(loans_data_unpacked)
    #print(type(feature))
    #loans_data_unpacked = pd.DataFrame(data = loans_data_unpacked)
    #print(loans_data_unpacked)
#     dfx = pd.data()

    # Change None's to 0's
    
    del loans_data[feature]
    loans_data = pd.concat([loans_data, loans_data_unpacked],axis = 1)

In [65]:
categorical_variables

['grade', 'sub_grade', 'home_ownership', 'purpose', 'term']

In [74]:
train_data = loans_data.ix[train]
validation_data = loans_data.ix[validation]
# train_data = loans_data.iloc[train]
# validation_data = loans_data.iloc[validation]
#loans_data = risky_loans.append(safe_loans)
#loans_data
#loans_data.append(loans_data_unpacked)
#loans_data_unpacked.dtypes
#loans_data_new = loans_data['short_emp']
#pd.concat([loans_data_new, loans_data_unpacked], axis = 1)

In [61]:
train[-10:]

[122552,
 122553,
 122558,
 122561,
 122569,
 122572,
 122575,
 122588,
 122599,
 122603]

In [68]:
#df = pd.DataFrame(data = loans_data_one_hot_encoded)
#unpack(df,'grade',0)
print (1)

1


9. Now, let's use the built-in scikit learn decision tree learner (sklearn.tree.DecisionTreeClassifier) to create a loan prediction model on the training data. To do this, you will need to import sklearn, sklearn.tree, and numpy.

Note: You will have to first convert the SFrame into a numpy data matrix, and extract the target labels as a numpy array (Hint: you can use the .to_numpy() method call on SFrame to turn SFrames into numpy arrays). See the API for more information. Make sure to set max_depth=6.

Call this model decision_tree_model.

10. Also train a tree using with max_depth=2. Call this model small_model.

In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
decision_tree_model = DecisionTreeClassifier(max_depth = 6)
small_model = DecisionTreeClassifier(max_depth = 2)

In [75]:
X = np.array(train_data)
Y = np.array(train_data[target])
decision_tree_model = decision_tree_model.fit(X,Y)
small_model = small_model.fit(X,Y)

In [76]:
decision_tree_model.n_features_

68

In [251]:
from sklearn import tree
i_tree = 0
for tree_in_forest in forest.estimators_:
    with open('tree_' + str(i_tree) + '.dot', 'w') as my_file:
        my_file = tree.export_graphviz(tree_in_forest, out_file = my_file)
    i_tree = i_tree + 1

In [78]:
from sklearn import tree
X = [[0, 0], [1, 1]]
Y = [0, 1]
from sklearn.externals.six import StringIO  
import pydot 
dot_data = StringIO() 
tree.export_graphviz(small_model, out_file=dot_data) 
graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
#graph.write_pdf("iris.pdf") 

In [240]:
from IPython.display import Image  
dot_data = StringIO()  
tree.export_graphviz(small_model, out_file=dot_data,  
                         feature_names=iris.feature_names,  
                         class_names=iris.target_names,  
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = pydot.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())  

NameError: name 'iris' is not defined

In [242]:
from sklearn.datasets import load_iris
iris = load_iris()

In [79]:
import graphviz
from pandas import read_csv, DataFrame
from sklearn import tree
from os import system

# data = read_csv('D:/training.csv')
# Y = data.Y
# X = data.ix[:,"X0":"X33"]

# dtree = tree.DecisionTreeClassifier(criterion = "entropy")
# dtree = dtree.fit(X, Y)

import pydotplus
dotfile = StringIO()
tree.export_graphviz(decision_tree_model, out_file=dotfile)
graph = pydot.graph_from_dot_data(dotfile.getvalue())
graph[0].write_pdf("iris.pdf") 

Exception: "dot.exe" not found in path.

In [80]:
clf = clf.fit(iris.data, iris.target)
tree.export_graphviz(decision_tree_model,out_file='tree1.dot') 

NameError: name 'clf' is not defined